<h1> FLOW<h2>
<h3>> Certain amount of time is set to give use some time to get ready</h3>
<h3>> the trainer will then assume the pose that leads to the first part of the exercise</h3>
<h3>> an indicator will prompt the user to perform the exercise</h3>
<h3>> the user will have to stop in between exercise</h3>

    example:    
        push-up:    
            1) Starting Position:
            2) Push-Up Descent:
            3) Push-Up Ascent:  
            4) Stop momentarily, then it'll prompt to do it again   

<h3>> every start of the exercise will be recorded unto a txt file for the landmark coordinates</h3>


        

data cleaning game plan
>collect data
>apply first outlier detection, common sequences(sequences that has the most frequency)
>apply second outlier detection, z-score

cons for all of these approach
> may delete vital data(but can be mitigated with careful determining of threshold)
> reduces the dataset significantly(but can be solved through data augmentation)

-------------------------------------------------------------------------
data augmentation game plan
> cleaned data is introduced to padding and truncating with noise instead of just 0
>apply padding both post and pre(add pre-padding and post padding until the last sequence reaches the last indext of the max legnth) = this would simulate the constant stream of data and predicting(previous iteration was only paost padding resulting into total dependency whether the first sequence of the exercise is somewhere at the first index...this creates numerous problem.it would not predict as correct if the sequence is at the end portion of the index or in the middle in some cases)...adding noise would also (theoretically) make the model more robust
>adding noises through feeding incorrect

>we may also consider including the outlier that have more and less sequence(less sequence will be stretched out or the data will be disperesed with certain threshold this would mean that all data will be evenly distributed along the ideal number of sequence or the max sequence and add 0 or noise in between sequences to fill in the missing values...on the other hand, more sequences, we may opt to eliminate some of the data in between randomly until it reaches the max legnth or the ideal legnth)....however at the moment, i think this approach may create more problems in training due to the nature of its augmentations that may not totally represent the correct sequences(since we are already heavily augmenting it).

>in general we may need to augment our data that simulate cases that may occur in inferencing
>scenarios:
    >certain vital part of the sequence may not have been capture due to some reason(frame skipped or we only collect data after every frame)
    >constant stream of data being appended unto the array resulting to noise....supposed that: VS == vital seeuqnce and NS == noise sequence example-> [NS,NS,NS,VI,VI,VI,VI,NS,NS]
    > vital sequence is in between noise sequences therefore the usual padding may not be sufficient and may not represent the kind of seuqnce under noise previous padding approach->[VS,VS,VS,VS,0,0,0,0] it has a total dependecy whether it the sequence is at the left most or the first index therfore it may not come up as a correct sequence if it have not arrived at the very first index.
    >REMEMBER!-----------------------------> IT MUST NOT EXCEED a maximum of 50% of the total length since it might

ADD A PURGING MECHANIC(DO NOT FORGET!)
> it would clear the array if it had successfully predicted a correct
> this would prevent situtaions wherein 2 correct sequences will be in the array



problem encountered -> certain stance(if u dont move would be considered a correct or one exercise)
> current hypothesis to this problem is that of how the data was augmented in the first place. the correct data was augmented to stride along the expanded length therefore now matter where the sequence is situated in the array, it will be considered correct, it also provides more flexibility and leeway. Model might have interpeted that commonality and pattern as all the sequence stride along the array.
> this problem may be solved by simulating this scenario. through augmenting the correct data to have this scenario and claissfy as correct, this might be able to train the model to have this kind of scenario to be classified as falls.


    >for instance : suppose array = [[1],[2],[3],[4],] will stride along an array with a length of 7
    [0,0,0,[1],[2],[3],[4],]
    [0,0,[1],[2],[3],[4],0]
    [0,[1],[2],[3],[4],0,0,]
    [[1],[2],[3],[4],0,0,0,]

    model may interpret [,[1],[1],[1],[1],[1],[1],[1]] as correct


In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model
import tensorflow as tf
from keras.callbacks import Callback
from tensorflow.keras.regularizers import l1, l2
import random as rand

import numpy as np
import cv2
import mediapipe as mp
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

# from tensorflow_model_optimization.quantization.keras import quantize_model
from collections import Counter
from IPython.display import clear_output
import os

In [35]:
# # DATA COLLECTING
# import cv2
# import mediapipe as mp



# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
# mp_drawing = mp.solutions.drawing_utils

# # cap = cv2.VideoCapture("vid4.mp4")
# cap = cv2.VideoCapture(0)


# # PARAMETERS:
# # ================================================================================================================================
# # timer to ready before posing/getting ready for the excercise
# set_timer_start = 50

# # ================================================================================================================================
# # range of how much change will it be considered a movement
# # lower -> small movement and jittery flaw of mediapipe will be detected
# # higher -> it wont detect immediately important frames
# # 0-1
# change_range = .03

# # ================================================================================================================================
# # a counter for the amount of time for the wait time before performing the exercise again
# between_exercise_ctr = 10



# isInitiated = False
# exercise_exectuted_ctr=0
# prev_x = 0
# prev_y = 0
# no_movement_ctr = 0
# movement_boolean = False
# timer = 0
# no_movement_threshold = 32

# prev_array = []


# for x in range(33):
#     prev_array.append([0, 0])

# while True:
#     ret, frame = cap.read()
#     frame = cv2.flip(frame, 1)
#     image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = pose.process(image_rgb)

#     if not ret:
#         break

#     timer += 1 
#     landmark_all_present = False

#     if timer == set_timer_start:
#         file = open('base1234.txt', 'w')
#         if isInitiated==False:
#             file.write("START")
#             file.write('\n')
#         isInitiated = True

#     if results.pose_landmarks:
#         ctr_check=0

#         for idx, landmark in enumerate(results.pose_landmarks.landmark):
#             if idx == 32:
#                 landmark_all_present = "Landmarks -> ALL PRESENT"
#             else:
#                 landmark_all_present = "Landmarks -> SOME ARE MISSING"

#             x = landmark.x
#             y = landmark.y

# #checks if no movement is detected-----------------------------------------------------------------------------------------------------------------------------------------
#             if x >= (prev_array[idx][0] - change_range) and y <= (prev_array[idx][1] + change_range) and isInitiated:   
#                 print(idx,")",x,"---",prev_array[idx][0],'---> NO MOVEMENT')
#                 ctr_check+=1          
                
#             else:
#                 print(idx,")",x,"---",prev_array[idx][0],'--->YES MOVEMENT',)
#                 movement_boolean = True
#                 no_movement_ctr = 0

#             if ctr_check == no_movement_threshold:
#                 no_movement_ctr += 1 

#             if no_movement_ctr>=25:
#                 movement_boolean=False
#             prev_array[idx] = [x, y]
#         print('counter->',no_movement_ctr)
#         print(movement_boolean)

# #if movement is detected then it will record the coordinates into a text file-----------------------------------------------------------------------------------------------------
#         if movement_boolean:
#             text = "MOVEMENT"      
#             if timer >= set_timer_start and file is not None:
#                 for landmark in prev_array:
#                     # file.write("<")
#                     file.write("|")
#                     file.write(str(landmark[0]))
#                     file.write("|")
#                     file.write(str(landmark[1]))
#                     # file.write(">")
#                     # file.write("|")
#                 file.write('\n')

#         if no_movement_ctr >= 5:    
#             movement_boolean = False
#             text = "NO MOVEMENT!"

#         if no_movement_ctr == between_exercise_ctr:
#             exercise_exectuted_ctr+=1
#             file.write("END")  
#             file.write('\n')
#             file.write('\n')
#             file.write("START")
#             file.write('\n')



#         if timer >= set_timer_start and no_movement_ctr >=between_exercise_ctr:
#             cv2.putText(frame, "PERFORM THE EXERCISE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
# # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# #Outputing important informations-----------------------------------------------------------------------------------------------------------------------------------------------
#         cv2.putText(frame, str(movement_boolean), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, f"no_movement_ctr-> {no_movement_ctr}", (20, 37), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, str(landmark_all_present), (20, 54), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, f"timer-> {timer}", (20, 73), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, f"exercise exectuted-> {exercise_exectuted_ctr}", (20, 92), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
# # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#         if timer < set_timer_start:
#             cv2.putText(frame, "PREPARE YOUR STARTING POSE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
            

#         mp_drawing.draw_landmarks(
#             frame,
#             results.pose_landmarks,
#             mp_pose.POSE_CONNECTIONS,
#             landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
#             connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
#         )

#     cv2.imshow('MediaPipe Pose', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# if file is not None:
#     file.close()
# cap.release()
# cv2.destroyAllWindows()


In [36]:
# Functions
def txt_pre_process(txt_file,label,simplify=False,simplify_level=4 ):
    label_array = []
    temp_feature_data = []
    temp_sequence_data = []
    batch_data = []

    with open(str(txt_file), 'r') as file:

        for line in file:
            values = line.strip().split('|')        

            temp_feature_data = []

            for value in values:            
                float_value = str(value)

                #FIRST PART OF THE SEQUENCE 
                if float_value == 'START':   
                    temp_sequence_data=[]
                                        
                elif float_value == 'END':              
                    batch_data.append(temp_sequence_data)
                    label_array.append(label)

            
                elif float_value != '' and float_value != 'START': 
                    if simplify:    
                        float_value = round(float(value),simplify_level)
                    else:
                        float_value = float(value)
                    temp_feature_data.append(float_value)

            if temp_feature_data!=[]:
                temp_sequence_data.append(temp_feature_data)

    label_array = np.array(label_array)
    return [batch_data,label_array]

#--------------------------------------------------------------------------- paddingV1 --------------------------------------------------------------------------------
# padding can be improved probably...by using sequence 
# minor issue:
# > is whether sequences had exceeded the intended number of sequences but is still right (it was performed right but slower(by an acceptable margin)) - not resolved
#    = temporary fix was just to truncate everything if it had exceeded the intended number of sequence for the sake of running it for now
#    = a reliable solution in theory could be that to randomly truncate in between the first and end sequence, in this way relevant data can be captured
def padding(pre_processed_input,optional_maxLength=0): 
    padded_sequences = []
    if optional_maxLength != 0:
        max_length = optional_maxLength
    else:
        max_length = max(len(sequence) for sequence in pre_processed_input)
    
    for sequence in pre_processed_input:        
        padding_length = max_length - len(sequence)
        if padding_length >= 0:
            padded_sequence = np.pad(sequence, ((0, padding_length), (0, 0)), mode='constant')
            
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    padded_sequences = np.array(padded_sequences)

    return padded_sequences

#--------------------------------------------------------------------------- paddingV1 --------------------------------------------------------------------------------

# this is to merge correct executions and wrong executions and randomize their input and label
# positions of input and its corresponding label are the same 
# introducing noise/wrong input makes the model more robust
def concatenate_randomize_batches(base_input,base_label,concat_input,concat_label):
    combined_inputs = np.concatenate((base_input,concat_input), axis = 0)
    combined_label = np.concatenate((base_label,concat_label), axis = 0)
    indices = np.random.permutation(len(combined_inputs))
    randomized_inputs = combined_inputs[indices]
    randomized_label = combined_label[indices]
    return [randomized_inputs,randomized_label]




# do not touch....for now...
# this is for assigning of correct input and incorrect input for testing later
def simulate_testing(dataset,index):   
    input_data_simulate = dataset
    max_sequence_length = 31

    simulate_abrupt = 31
    execution_index = index

    testing123=input_data_simulate[execution_index].reshape((1, 31, 66))
    temp69 = testing123[0][:simulate_abrupt]
    temp69 = temp69.reshape((1, simulate_abrupt, 66))
    temp69_padded = padding(temp69,31)
    print(model.predict(temp69_padded))

    correct_data =[]
    wrong_data =[]
    for x in range(len(predict_simulate_input)):
        if predict_simulate_label[x] == 1:
            correct_data.append(predict_simulate_input[x])
            simulate_testing(predict_simulate_input,x)
            print("CORRECT")
        else:
            wrong_data.append(predict_simulate_input[x])
            simulate_testing(predict_simulate_input,x)
            print("WRONG")

    correct_data = np.array(correct_data)
    wrong_data = np.array(wrong_data)
    print(correct_data.shape)
    print(wrong_data.shape) 



def tally_sequence(sequence_array):
    tally_number = []
    tally_ctr = []

    for x in sequence_array:
        temp = len(x)
        if temp not in tally_number:
            tally_number.append(temp)
            tally_ctr.append(1)
        else:
            for y in range(len(tally_number)) :
                if temp == tally_number[y]:
                    tally_ctr[y] = tally_ctr[y] + 1

    tally_max = 0
    tally_number_arranged = []
    tally_ctr_arranged = []

    for x in range(len(tally_number)):
        # print(len(tally_ctr)) 
        tally_max = max(tally_ctr)
        for y in range(len(tally_number)):
            if tally_ctr[y] == tally_max:
                tally_number_arranged.append(tally_number[y])
                tally_ctr_arranged.append(tally_ctr[y])                   
                tally_ctr.pop(y)
                tally_number.pop(y)
                break

    total_ctr = 0
    for x in tally_ctr:
        total_ctr = total_ctr + x


    for x in range(len(tally_number_arranged)):
        print(tally_number_arranged[x],'-->',tally_ctr_arranged[x])


# outlier detection and removal (currently being used)
def common_length_sequence(sequences_array,threshold = 2):
    temp = []

    data = [len(seq) for seq in sequences_array]
    data_frequency = Counter(data)
    most_common_data = data_frequency.most_common()
    outlier_frequencies = [value for value, freq in data_frequency.items() if freq < threshold]
    most_common_values = [value for value, freq in most_common_data if freq >= threshold]

    print("Most Common Data Points:", most_common_values)
    print("Outlier Frequencies:", outlier_frequencies)

    for x in sequences_array:        
        if len(x) in most_common_values:
            temp.append(x)
    print('-------------------applied frequency outlier detection-------------------')
    print("original num -> ", len(sequences_array))
    print("current num -> ", len(temp))
    print("removed num -> ", len(sequences_array) - len(temp))
    return temp

# outlier detection and removal (currently being used)
def apply_z_score(sequences_array,z_score_threshold = 1):
    data_points = []
    included_datapoints = []
    updated_sequences =[]

    for x in sequences_array:
        temp = len(x)
        if temp not in data_points:
            data_points.append(temp)

    data = np.array(data_points)
    mean_value = np.mean(data)
    standard_deviation = np.std(data)
    z_scores = (data - mean_value) / standard_deviation
    for x in range(len(z_scores)):
        if np.abs(z_scores[x]) <= z_score_threshold:
            included_datapoints.append(data[x])


    for x in sequences_array:
        if len(x) in included_datapoints:
            updated_sequences.append(x)
    print('-------------------applied z-score outlier detection-------------------')
    print("datapoints included -> ", included_datapoints)
    print("original num -> ", len(sequences_array))
    print("current num -> ", len(updated_sequences))
    print("removed num -> ", len(sequences_array) - len(updated_sequences))

    return updated_sequences




def paddingV2(sequences_array,optional_maxlength = 0):
    output = []
    max_length = 0
    if optional_maxlength == 0:
        max_length = max(len(sequence) for sequence in sequences_array)
        expanded_max_length = int(max_length+ ((max_length) * .40))
    else:
        expanded_max_length = optional_maxlength

    print(expanded_max_length)


    padding_length_before = 0
    padding_length_after = 0

    for seq in sequences_array:
        # print(seq)
        for x in range(expanded_max_length-len(seq)+1):
            padding_length_before = x
            padding_length_after = expanded_max_length - len(seq) - x
            padded_sequence = np.pad(seq, ((padding_length_before, padding_length_after),(0,0)), mode='constant')
            output.append(padded_sequence)
            
            # print(padded_sequence)
    print('------------------------applied paddingV2------------------------')
    print('max_length -> ', max_length)
    print('expanded_max_length -> ', expanded_max_length)
    print('original num set of sequences -> ', len(sequences_array))
    print('final num set of sequences -> ', len(output))

    output = np.array(output)
    return output


def populate_0_input(correct_data,noise_data):
    print(len(correct_data))
    index = 10
    temp = []
    temp_compilation = []
    ctr = 0
    rand_modifier =0
    
    for set_sequence in correct_data:
        rand_modifier = rand.randint(0,len(noise_data))

        for x in range(len(set_sequence)):
            ctr = ctr + 1 
            if set_sequence[x][0] == 0:
                temp.append(noise_data[rand_modifier-1][rand.randint(0,len(noise_data[rand_modifier-1])-1)])

            else:
                temp.append(set_sequence[x])

        temp_compilation.append(temp)
        temp =[]


    return temp_compilation





def convert_tf_to_tflite(tf_model):
  model = tf.keras.models.load_model(tf_model)

  run_model = tf.function(lambda x: model(x))
  # This is important, let's fix the input size.
  BATCH_SIZE = 1
  STEPS = 31
  INPUT_SIZE = 66
  concrete_func = run_model.get_concrete_function(
      tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

  # model directory.
  MODEL_DIR = "keras_lstm"
  model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

  converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
  
  Converter.from_saved_model(MODEL_DIR)
  tflite_model = converter.convert()


  # Run the model with TensorFlow to get expected results.
  TEST_CASES = 10

  # Run the model with TensorFlow Lite
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  for i in range(TEST_CASES):
    expected = model.predict(X_test[i:i+1])
    interpreter.set_tensor(input_details[0]["index"], X_test[i:i+1, :, :])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])


    np.testing.assert_almost_equal(expected, result, decimal=5)
    print("it matches!!!!!!!!!.")

    interpreter.reset_all_variables()



def model_inference(input,model,input_shape,input_details,output_details,name):
    prediction = np.array(input)
    prediction = prediction.astype(np.float32)
    prediction = prediction.reshape(1,input_shape[1],input_shape[2])
    model.set_tensor(input_details[0]['index'], prediction)
    model.invoke()            
    output_data = model.get_tensor(output_details[0]['index'])
    print(name,'--->',output_data[0][0])
    return(output_data[0][0])


def draw_pose_lines(image,coordinates, color=(255, 0, 0), thickness=5):   

    start_point = (int(coordinates[0] * image.shape[1]), 
                    int(coordinates[1] * image.shape[0]))
    end_point = (int(coordinates[2] * image.shape[1]), 
                    int(coordinates[3]* image.shape[0]))

    # Draw the line on the image
    cv2.line(image, start_point, end_point, color, thickness)


class CustomEarlyStopping(Callback):
  def __init__(self, accuracy_threshold=0.95, loss_threshold=0.10):
      super(CustomEarlyStopping, self).__init__()
      self.accuracy_threshold = accuracy_threshold
      self.loss_threshold = loss_threshold

  def on_epoch_end(self, epoch, logs=None):
      if logs is None:
          logs = {}

      if logs.get('val_accuracy') is None or logs.get('val_loss') is None:
          return

      if logs.get('val_accuracy') >= self.accuracy_threshold and logs.get('val_loss') <= self.loss_threshold:
          self.model.stop_training = True
          print(f"\nTraining stopped as validation accuracy reached {logs.get('val_accuracy'):.4f} "
                f"and validation loss reached {logs.get('val_loss'):.4f}")


def aug_noise(sequences_array,optional_maxlength = 0):
    output = []
    temp = []
    temp_array = []
    temp_random_list = []
    rand_temp = 0
    max_length = 0
    random_ctr = 0
    new_sequence_array=[]
    padding_length_before = 0
    padding_length_after = 0

    random_ctr_modifier = .5

    if optional_maxlength == 0:
        max_length = max(len(sequence) for sequence in sequences_array)
        expanded_max_length = int(max_length+ ((max_length) * .40))
    else:
        expanded_max_length = optional_maxlength

    random_ctr = int(len(sequences_array) *random_ctr_modifier)


    for rand_num in range(random_ctr):
        while rand_temp in temp_random_list:
            rand_temp = rand.randint(1,len(sequences_array))-1
        temp_random_list.append(rand_temp)

    for x in temp_random_list:
        new_sequence_array.append(sequences_array[x])

# first noise/incorrrect generation(if you freeze at certain stance):
    for set_of_sequence in new_sequence_array:
        for one_sequence in set_of_sequence:
            temp = []
            for ctr in range(expanded_max_length):
                temp.append(one_sequence)
            temp_array.append(temp)




In [37]:
# # this is as a whole exercise
# def streamlined_process(correct_execution,noise_data):
#   base_data = txt_pre_process(correct_execution,1,False,4)
#   base_data_noise = txt_pre_process(noise_data,0,False,4)

#   temp_correct_start=[]
#   temp_correct_wrong=[]

#   data = base_data[0]
#   data_noise = base_data_noise[0]

#   print(data[0])

#   print('')
#   print('----------------------------correct data augmentation ----------------------------')
#   aug = common_length_sequence(data)
#   aug2 = apply_z_score(aug,1)
#   aug3 = paddingV2(aug2)
#   aug4 = populate_0_input(aug3,data_noise)
#   aug4 = np.array(aug4)
#   aug4 = aug4.reshape(-1,len(aug3[0]),len(aug3[0][0]))
#   combined_inputs = np.concatenate((aug4,aug3), axis = 0)
#   combined_inputs = aug4
#   print('concat -> ', len(combined_inputs))isInitiated = True


#   print('')
#   print('----------------------------data noise data augmentation ----------------------------')
#   aug_noise_data1 = paddingV2(data_noise,len(aug3[0]))
#   aug_noise_data2 = populate_0_input(aug_noise_data1,data_noise)
#   aug_noise_data3 = aug_noise_data2[0:len(combined_inputs)]
#   aug_noise_data4 = np.array(aug_noise_data3)
#   aug_noise_data5 = aug_noise_data4.reshape(-1,len(aug_noise_data4[0]),len(aug_noise_data4[0][0]))


#   aug3_label = np.ones(len(combined_inputs))
#   aug_noise_label = np.zeros(len(aug_noise_data1))


#   rand_batches=concatenate_randomize_batches(combined_inputs,aug3_label,aug_noise_data5,aug_noise_label)

#   X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.3, random_state=42)

# # basis in this version of model is the amount of sequences ( do not delete this! this is working!!!)--------------------------------------------------------------------------------------
#   # model = Sequential()
#   # model.add(Bidirectional(LSTM(len(aug3[0]), return_sequences=True, activation='relu',  input_shape=(len(aug3[0]), len(aug3[0][0])))))
#   # model.add(Bidirectional(LSTM(len(aug3[0]) + int(len(aug3[0]) - int(len(aug3[0]) * .5)), return_sequences=True ,kernel_regularizer=l2(0.001),  activation='relu')))
#   # model.add(Bidirectional(LSTM(len(aug3[0]) - int(len(aug3[0]) - int(len(aug3[0]) * .3)), return_sequences=True, dropout=0.1, recurrent_dropout=0.1, activation='relu')))
#   # model.add(Bidirectional(LSTM(len(aug3[0]) - int(len(aug3[0]) - int(len(aug3[0]) * .5)), return_sequences=False, kernel_regularizer=l2(0.001), activation='relu')))
#   # model.add(Dense(len(aug3[0]) - int(len(aug3[0]) - int(len(aug3[0]) * .5)), activation='relu'))
#   # model.add(Dense(1,activation='sigmoid'))
#   # model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])
#   # model.fit(X_train, y_train, epochs=100, batch_size =128 , validation_data=(X_test, y_test))
# # -------------------------------(DO NOT DELETE! THIS IS A WORKING ARCHITECTURE)-------------------------------------------------------------------------------------------------------------------



# # yields better performance(DO NOT DELETE BETTER VERSION)-------------------------------------------------------------------------------
#   model = Sequential()
#   model.add(Bidirectional(LSTM(len(aug3[0]), return_sequences=True, activation='relu',  input_shape=(len(aug3[0]), len(aug3[0][0])))))
#   model.add(Bidirectional(LSTM(len(aug3[0]) + int(len(aug3[0]) - int(len(aug3[0]) * .5)), return_sequences=True ,kernel_regularizer=l2(0.01),  activation='relu')))
#   model.add(Bidirectional(LSTM(len(aug3[0]) - int(len(aug3[0]) - int(len(aug3[0]) * .2)), return_sequences=True, dropout=0.1, recurrent_dropout=0.1, activation='relu')))
#   model.add(Bidirectional(LSTM(len(aug3[0]) - int(len(aug3[0]) - int(len(aug3[0]) * .5)), return_sequences=False, kernel_regularizer=l2(0.01), activation='relu')))
#   model.add(Dense(len(aug3[0]) - int(len(aug3[0]) - int(len(aug3[0]) * .4)), activation='relu'))
#   model.add(Dense(1,activation='sigmoid'))
#   model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])
#   model.fit(X_train, y_train, epochs=150, batch_size =128 , validation_data=(X_test, y_test))
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------





#   # basis in this version of model is the amount of features per sequences -----------------------------------------------------------------------------------------
#   # model = Sequential()
#   # model.add(Bidirectional(LSTM(len(aug3[0][0]), return_sequences=True, activation='relu',  input_shape=(len(aug3[0]), len(aug3[0][0])))))
#   # model.add(Bidirectional(LSTM(len(aug3[0][0]) + int(len(aug3[0][0]) - int(len(aug3[0][0]) * .6)), return_sequences=True ,kernel_regularizer=l2(0.01),  activation='relu')))
#   # model.add(Bidirectional(LSTM(len(aug3[0][0]) - int(len(aug3[0][0]) - int(len(aug3[0][0]) * .4)), return_sequences=True, dropout=0.1, recurrent_dropout=0.1, activation='relu')))
#   # model.add(Bidirectional(LSTM(len(aug3[0][0]) - int(len(aug3[0][0]) - int(len(aug3[0][0]) * .6)), return_sequences=False, kernel_regularizer=l2(0.01), activation='relu')))
#   # model.add(Dense(len(aug3[0][0]) - int(len(aug3[0]) - int(len(aug3[0][0]) * .6)), activation='relu'))
#   # model.add(Dense(1,activation='sigmoid'))
#   # model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])
#   # model.fit(X_train, y_train, epochs=135, batch_size =128 , validation_data=(X_test, y_test))
# # ----------------------------------------------------------------------------------------


#   model.save('testingModel')


#   X_train = X_train.astype(np.float32)
#   X_test = X_test.astype(np.float32)



# # update this....the parameter increased
#   convert_tf_to_tflite('/content/testingModel',[1,len(aug3[0]),len(aug3[0][0])], X_test)



#   # return(aug3,aug_noise)





# # streamlined_process('base(correct_execution).txt','base(wrong_execution).txt')



In [38]:

# 0 1  -> 0
# 2 3  -> 1
# 4 5  -> 2
# 6 7  -> 3
# 8 9  -> 4
# 10 11  -> 5
# 12 13  -> 6
# 14 15  -> 7
# 16 17  -> 8
# 18 19  -> 9
# 20 21  -> 10
# 22 23  -> 11
# 24 25  -> 12
# 26 27  -> 13
# 28 29  -> 14
# 30 31  -> 15
# 32 33  -> 16
# 34 35  -> 17
# 36 37  -> 18
# 38 39  -> 19
# 40 41  -> 20
# 42 43  -> 21
# 44 45  -> 22
# 46 47  -> 23
# 48 49  -> 24
# 50 51  -> 25
# 52 53  -> 26
# 54 55  -> 27
# 56 57  -> 28
# 58 59  -> 29
# 60 61  -> 30
# 62 63  -> 31
# 64 65  -> 32

# # ----------------------------------------------------------------------------
#   left_upper_arm_sequence = []
#   left_lower_arm_sequence = []
#   left_hand_sequence = []

#   right_upper_arm_sequence = []
#   right_lower_arm_sequence = []
#   right_hand_sequence = []

#   left_upper_leg_sequence = []
#   left_lower_leg_sequence = []
#   left_feet_sequence = []

#   right_upper_leg_sequence = []
#   right_lower_leg_sequence = []
#   right_feet_sequence = []

#   body_sequence = []
#   head_sequence = []

#   # ------------------------------------------------------------------------------
#   left_upper_arm_sequence_noise = []
#   left_lower_arm_sequence_noise = []
#   left_hand_sequence_noise = []

#   right_upper_arm_sequence_noise = []
#   right_lower_arm_sequence_noise = []
#   right_hand_sequence_noise = []

#   left_upper_leg_sequence_noise = []
#   left_lower_leg_sequence_noise = []
#   left_feet_sequence_noise = []

#   right_upper_leg_sequence_noise = []
#   right_lower_leg_sequence_noise = []
#   right_feet_sequence_noise = []

#   body_sequence_noise = []
#   head_sequence_noise = []




In [39]:
# # this is for the individual models
# def streamlined_process(correct_execution,noise_data):
#   id_num = str(rand.randint(1000,9999))
#   base_data = txt_pre_process(correct_execution,1,False,4)
#   base_data_noise = txt_pre_process(noise_data,0,False,4)

#   temp_correct_start=[]
#   temp_correct_wrong=[]

#   data = base_data[0]
#   data_noise = base_data_noise[0]



#   print('')
#   print('----------------------------correct data augmentation ----------------------------')
#   aug = common_length_sequence(data)
#   aug2 = apply_z_score(aug,1)
#   aug3 = paddingV2(aug2)
#   aug4 = populate_0_input(aug3,data_noise)
#   aug4 = np.array(aug4)
#   aug4 = aug4.reshape(-1,len(aug3[0]),len(aug3[0][0]))
#   combined_inputs = np.concatenate((aug4,aug3), axis = 0)
#   print('concat -> ', len(combined_inputs))


#   print('')
#   print('----------------------------data noise data augmentation ----------------------------')
#   aug_noise_data1 = paddingV2(data_noise,len(aug3[0]))
#   aug_noise_data2 = populate_0_input(aug_noise_data1,data_noise)
#   aug_noise_data3 = aug_noise_data2[0:len(combined_inputs)]
#   aug_noise_data4 = np.array(aug_noise_data3)
#   aug_noise_data5 = aug_noise_data4.reshape(-1,len(aug_noise_data4[0]),len(aug_noise_data4[0][0]))


# # initializing temp storage for data


#   correct_data_set = []
#   noise_data_set = []
#   rand_batches = []


#   for x in range(14):
#     correct_data_set.append([])
#     noise_data_set.append([])
#     rand_batches.append([])


#   for exercise in combined_inputs:


#     left_upper_arm = []
#     left_lower_arm = []
#     left_hand = []
#     right_upper_arm = []
#     right_lower_arm = []
#     right_hand = []
#     left_upper_leg = []
#     left_lower_leg = []
#     left_feet = []
#     right_upper_leg = []
#     right_lower_leg = []
#     right_feet = []
#     head =[]
#     body = []



# #------------------------ generating the correct input of certain part------------------------------
#     for sequence in exercise:
#       # 11,13
#       left_upper_arm.append([sequence[22],sequence[23],sequence[26],sequence[27]])
#       # 13,15
#       left_lower_arm.append([sequence[26],sequence[27],sequence[30],sequence[31]])
#       # 15,17,19,21
#       left_hand.append([sequence[30],sequence[31],sequence[34],sequence[35],sequence[38],sequence[39],sequence[42],sequence[43]])

#       # 12,14
#       right_upper_arm.append([sequence[24],sequence[25],sequence[28],sequence[29]])
#       # 14,16
#       right_lower_arm.append([sequence[28],sequence[29],sequence[32],sequence[33]])
#       # 16,18,20,22
#       right_hand.append([sequence[32],sequence[33],sequence[36],sequence[37],sequence[40],sequence[41],sequence[44],sequence[45]])

#       # 23,25
#       left_upper_leg.append([sequence[46],sequence[47],sequence[50],sequence[51]])
#       # 25,27
#       left_lower_leg.append([sequence[50],sequence[51],sequence[54],sequence[55]])
#       # 27,29,31
#       left_feet.append([sequence[54],sequence[55],sequence[58],sequence[59],sequence[62],sequence[63]])

#       # 24,26
#       right_upper_leg.append([sequence[48],sequence[49],sequence[52],sequence[53]])
#       # 26,28
#       right_lower_leg.append([sequence[52],sequence[53],sequence[56],sequence[57]])
#       # 28,30,32
#       right_feet.append([sequence[56],sequence[57],sequence[60],sequence[61],sequence[64],sequence[65]])

#       # 11,12,23,24
#       body.append([sequence[22],sequence[23],sequence[24],sequence[25],sequence[46],sequence[47],sequence[48],sequence[49]])
#       # 7,8,9,10
#       head.append([sequence[14],sequence[15],sequence[16],sequence[17],sequence[18],sequence[19],sequence[20],sequence[21]])


#     correct_data_set[0].append(left_upper_arm)
#     correct_data_set[1].append(left_lower_arm)
#     correct_data_set[2].append(left_hand)

#     correct_data_set[3].append(right_upper_arm)
#     correct_data_set[4].append(right_lower_arm)
#     correct_data_set[5].append(right_hand)

#     correct_data_set[6].append(left_upper_leg)
#     correct_data_set[7].append(left_lower_leg)
#     correct_data_set[8].append(left_feet)

#     correct_data_set[9].append(right_upper_leg)
#     correct_data_set[10].append(right_lower_leg)
#     correct_data_set[11].append(right_feet)

#     correct_data_set[12].append(body)
#     correct_data_set[13].append(head)




#   for exercise in aug_noise_data5:


#     left_upper_arm = []
#     left_lower_arm = []
#     left_hand = []
#     right_upper_arm = []
#     right_lower_arm = []
#     right_hand = []
#     left_upper_leg = []
#     left_lower_leg = []
#     left_feet = []
#     right_upper_leg = []
#     right_lower_leg = []
#     right_feet = []
#     head =[]
#     body = []


# #------------------------------ generating noise ---------------------------------------
#     for sequence in exercise:
#       # 11,13
#       # print(sequence[22])
#       left_upper_arm.append([sequence[22],sequence[23],sequence[26],sequence[27]])
#       # 13,15
#       left_lower_arm.append([sequence[26],sequence[27],sequence[30],sequence[31]])
#       # 15,17,19,21
#       left_hand.append([sequence[30],sequence[31],sequence[34],sequence[35],sequence[38],sequence[39],sequence[42],sequence[43]])

#       # 12,14
#       right_upper_arm.append([sequence[24],sequence[25],sequence[28],sequence[29]])
#       # 14,16
#       right_lower_arm.append([sequence[28],sequence[29],sequence[32],sequence[33]])
#       # 16,18,20,22
#       right_hand.append([sequence[32],sequence[33],sequence[36],sequence[37],sequence[40],sequence[41],sequence[44],sequence[45]])

#       # 23,25
#       left_upper_leg.append([sequence[46],sequence[47],sequence[50],sequence[51]])
#       # 25,27
#       left_lower_leg.append([sequence[50],sequence[51],sequence[54],sequence[55]])
#       # 27,29,31
#       left_feet.append([sequence[54],sequence[55],sequence[58],sequence[59],sequence[62],sequence[63]])

#       # 24,26
#       right_upper_leg.append([sequence[48],sequence[49],sequence[52],sequence[53]])
#       # 26,28
#       right_lower_leg.append([sequence[52],sequence[53],sequence[56],sequence[57]])
#       # 28,30,32
#       right_feet.append([sequence[56],sequence[57],sequence[60],sequence[61],sequence[64],sequence[65]])

#       # 11,12,23,24
#       body.append([sequence[22],sequence[23],sequence[24],sequence[25],sequence[46],sequence[47],sequence[48],sequence[49]])
#       # 7,8,9,10
#       head.append([sequence[14],sequence[15],sequence[16],sequence[17],sequence[18],sequence[19],sequence[20],sequence[21]])


#     noise_data_set[0].append(left_upper_arm)
#     noise_data_set[1].append(left_lower_arm)
#     noise_data_set[2].append(left_hand)

#     noise_data_set[3].append(right_upper_arm)
#     noise_data_set[4].append(right_lower_arm)
#     noise_data_set[5].append(right_hand)

#     noise_data_set[6].append(left_upper_leg)
#     noise_data_set[7].append(left_lower_leg)
#     noise_data_set[8].append(left_feet)

#     noise_data_set[9].append(right_upper_leg)
#     noise_data_set[10].append(right_lower_leg)
#     noise_data_set[11].append(right_feet)

#     noise_data_set[12].append(body)
#     noise_data_set[13].append(head)


#   for x in range(len(correct_data_set)):
#     correct_data_set[x] = np.array(correct_data_set[x])

#   for x in range(len(noise_data_set)):
#     noise_data_set[x] = np.array(noise_data_set[x])


#   correct_data_set_label = np.ones(len(combined_inputs))
#   noise_data_set_label = np.zeros(len(aug_noise_data1))

#   for x in range(len(correct_data_set)):
#     rand_batches[x]=concatenate_randomize_batches(correct_data_set[x],correct_data_set_label,noise_data_set[x],noise_data_set_label)

#     # left_upper_arm = []
#     # left_lower_arm = []
#     # left_hand = []

#     # right_upper_arm = []
#     # right_lower_arm = []
#     # right_hand = []

#     # left_upper_leg = []
#     # left_lower_leg = []
#     # left_feet = []

#     # right_upper_leg = []
#     # right_lower_leg = []
#     # right_feet = []

#     # head =[]
#     # body = []


#   data_set_name=['left_upper_arm','left_lower_arm','left_hand','right_upper_arm','right_lower_arm','right_hand','left_upper_leg','left_lower_leg','left_feet','right_upper_leg','right_lower_leg','right_feet','head','body']
#   for x in range(len(data_set_name)-1):
#     print('progress -> ',x,'/',len(rand_batches[0][0]) )


#     X_train, X_test, y_train, y_test = train_test_split(rand_batches[x][0], rand_batches[x][1], test_size=0.3, random_state=42)



#     # model = Sequential()
#     # model.add(Bidirectional(LSTM(len(X_train[0]), return_sequences=True, activation='relu',  input_shape=(len(X_train[0]), len(X_train[0][0])))))
#     # model.add(Bidirectional(LSTM(len(X_train[0]) + int(len(X_train[0]) - int(len(X_train[0]) * .5)), return_sequences=True ,kernel_regularizer=l2(0.001),  activation='relu')))
#     # model.add(Bidirectional(LSTM(len(X_train[0]) - int(len(X_train[0]) - int(len(X_train[0]) * .3)), return_sequences=True, dropout=0.1, recurrent_dropout=0.1, activation='relu')))
#     # model.add(Bidirectional(LSTM(len(X_train[0]) - int(len(X_train[0]) - int(len(X_train[0]) * .5)), return_sequences=False, kernel_regularizer=l2(0.001), activation='relu')))
#     # model.add(Dense(len(X_train[0]) - int(len(X_train[0]) - int(len(X_train[0]) * .5)), activation='relu'))
#     # model.add(Dense(1,activation='sigmoid'))


#     model = Sequential()
#     model.add(LSTM(len(X_train[0]), return_sequences=True, activation='relu',  input_shape=(len(X_train[0]), len(X_train[0][0]))))
#     model.add(LSTM(len(X_train[0]) + int(len(X_train[0]) - int(len(X_train[0]) * .6)), return_sequences=True,  activation='relu'))
#     model.add(Bidirectional(LSTM(len(X_train[0]) - int(len(X_train[0]) - int(len(X_train[0]) * .4)), return_sequences=True, dropout=0.5, recurrent_dropout=0.5, activation='relu')))
#     model.add(LSTM(len(X_train[0]) - int(len(X_train[0]) - int(len(X_train[0]) * .5)), return_sequences=False,  activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dense(len(X_train[0]) - int(len(X_train[0]) - int(len(X_train[0]) * .4)), activation='relu'))
#     model.add(Dense(1,activation='sigmoid'))

#     custom_early_stopping = CustomEarlyStopping(accuracy_threshold=0.95, loss_threshold=0.07)
#     model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])
#     history = model.fit(X_train, y_train, epochs=200, batch_size =128 , validation_data=(X_test, y_test), callbacks=[custom_early_stopping])


#     val_loss = min(history.history['val_loss'])
#     val_accuracy = max(history.history['val_accuracy'])

#     # model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])
#     # model.fit(X_train, y_train, epochs=150, batch_size =128 , validation_data=(X_test, y_test))




#     model.save('testingModel')


#     X_train = X_train.astype(np.float32)
#     X_test = X_test.astype(np.float32)
#     convert_tf_to_tflite('/content/testingModel',[1,len(X_train[0]),len(X_train[0][0])], X_test,data_set_name[x],id_num,val_loss,val_accuracy)
#     clear_output(wait=True)




In [40]:
# streamlined_process('/content/drive/MyDrive/Colab Notebooks/correct_new_2.txt','/content/drive/MyDrive/Colab Notebooks/wrong_new_2.txt')

In [41]:
# base_data = txt_pre_process('base(correct_execution).txt',1,True,4)
# base_data_noise = txt_pre_process('base(wrong_execution).txt',0,True,4)

# data = base_data
# data_noise = base_data_noise

# print('')
# print('----------------------------correct data augmentation ----------------------------')
# aug = common_length_sequence(data)
# aug2 = apply_z_score(aug,1)
# aug3 = paddingV2(aug2)

# print('')
# print('----------------------------data noise data augmentation ----------------------------')
# aug_noise = paddingV2(data_noise,len(aug3[0]))
# aug_noise = aug_noise[0:len(aug3)]


# aug3_label = np.ones(len(aug3))
# aug_noise_label = np.zeros(len(aug_noise))


# rand_batches=concatenate_randomize_batches(aug3,aug3_label,aug_noise,aug_noise_label)
# print('')
# print('----------------------------final num set of sequences ----------------------------')
# print('correct data ->',len(aug3))
# print('wrong data ->',len(aug_noise))
# print('combined and randomized data ->',rand_batches[0].shape)




# X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.1, random_state=42)

# # further splitting the dataset to have a set for simulation later on
# X_train, predict_simulate_input, y_train, predict_simulate_label = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', dropout=0.1, recurrent_dropout=0.1, input_shape=(len(aug3[0]), len(aug3[0][0]))))
# model.add(LSTM(128, return_sequences=True, activation='relu')) 
# model.add(LSTM(64, return_sequences=False, activation='relu')) 
# model.add(Dense(64, activation='relu')) 
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])

# model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))



In [42]:
# model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

In [43]:
# model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


In [44]:
# # ---------CORRECT EXECUTION WITH SIMULATION OF ABRUPT STOP-----------------------
# # this is CORRECT EXECUTION but we will simulte a situation where the user will abruptly stop his/her execution
# # 31 is the max length of sequence - therefore each execution is 31 length

# # this variable simulates stoping at certain sequence

# execution_index = 1
# input_data_simulate = correct_data
# max_sequence_length = len(input_data_simulate[0])
# ctr = 0

# print("len of sequence",len(input_data_simulate[execution_index]))

# for x in correct_data[execution_index]:
#     ctr += 1
#     if x[0]==0:
#         break



# for x in range(max_sequence_length):
#     simulate_abrupt = x
# # choose between 2-30 and simulate abrupt stop or simulate where a user might stop in between execution


#     print(x,"/", ctr)
#     print("")




#     testing123=input_data_simulate[execution_index].reshape((1, 31, 66))
#     temp69 = testing123[0][:simulate_abrupt]
#     temp69 = temp69.reshape((1, simulate_abrupt, 66))
#     temp69_padded = padding(temp69,31)
#     print(model.predict(temp69_padded))

In [45]:
# # ---------WRONG EXECUTION WITH SIMULATION OF ABRUPT STOP-----------------------
# # this is WRONG EXECUTION but we will simulte a situation where the user will abruptly stop his/her execution
# # 31 is the max length of sequence - therefore each execution is 31 length

# # this variable simulates stoping at certain sequence
# input_data_simulate = wrong_data
# max_sequence_length = 31

# # choose between 2-30 and simulate abrupt stop or simulate where a user might stop in between execution
# simulate_abrupt = 31
# execution_index = 2

# testing123=input_data_simulate[execution_index].reshape((1, 31, 66))
# temp69 = testing123[0][:simulate_abrupt]
# temp69 = temp69.reshape((1, simulate_abrupt, 66))
# temp69_padded = padding(temp69,31)
# model.predict(temp69_padded)

In [46]:
# run_model = tf.function(lambda x: model(x))
# # This is important, let's fix the input size.
# BATCH_SIZE = 1
# STEPS = 31
# INPUT_SIZE = 66
# concrete_func = run_model.get_concrete_function(
#     tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# # model directory.
# MODEL_DIR = "keras_lstm"
# model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

# converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
# tflite_model = converter.convert()

In [47]:
# # Run the model with TensorFlow to get expected results.
# TEST_CASES = 10

# # Run the model with TensorFlow Lite
# interpreter = tf.lite.Interpreter(model_content=tflite_model)
# interpreter.allocate_tensors()
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# for i in range(TEST_CASES):
#   expected = model.predict(X_test[i:i+1])
#   interpreter.set_tensor(input_details[0]["index"], X_test[i:i+1, :, :])
#   interpreter.invoke()
#   result = interpreter.get_tensor(output_details[0]["index"])

#   # Assert if the result of TFLite model is consistent with the TF model.
#   np.testing.assert_almost_equal(expected, result, decimal=5)
#   print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

#   # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
#   # the states.
#   # Clean up internal states.
#   interpreter.reset_all_variables()

In [48]:
# # inferencing
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
# mp_drawing = mp.solutions.drawing_utils

# # cap = cv2.VideoCapture("vid4.mp4")
# cap = cv2.VideoCapture(0)
# frame_ctr = 0
# frame_ctr_threshold = 0
# exercise_ctr = 0
# prev_prediction = 0
# output_data = 0
# temp_checking = []
# all_array = []
# # it means it is waiting to reset
# prediction_buffer_threshold = .05
# model = tf.lite.Interpreter(model_path="D:\CLARK\Documents\RNN\RNN_proofOfConcept\converted_model8.tflite")
# model.allocate_tensors()
# input_details = model.get_input_details()
# output_details = model.get_output_details()
# input_shape = input_details[0]['shape']
# # print('input ->',)

# prediction =[]
# one_exercise = []
# # max length of sequence
# for y in range(input_shape[1]):
#     sequence_input=[]
#     # max num of landmark x and y flattened
#     for x in range(66):
#         sequence_input.append(0)
#     one_exercise.append(sequence_input)
    



# while True:
#     frame_ctr += 1
#     # print(frame_ctr)
#     ret, frame = cap.read()
#     frame = cv2.flip(frame, 1)
#     image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = pose.process(image_rgb)

#     if not ret:
#         break




#     if results.pose_landmarks:
#         temp_sequence=[]
#         for idx, landmark in enumerate(results.pose_landmarks.landmark):
#             x = landmark.x
#             y = landmark.y
#             # print(idx,")",x,"---",prev_array[idx][0],)
#             # print(idx)
#             x = round(float(x),4)
#             y = round(float(y),4)
#             temp_sequence.append(x)
#             temp_sequence.append(y)
#         one_exercise.append(temp_sequence)
#         print(one_exercise[0][0],'------',temp_sequence[0])
        

#     # if max length of the exercise sequence was reached then pop it pop off
#         if len(one_exercise) >= input_shape[1]:
#             one_exercise.pop(0)

#         all_array.append(one_exercise[:])
        
#         if frame_ctr >= frame_ctr_threshold:            
#             frame_ctr = 0
            
#             prediction = np.array(one_exercise)
#             prediction = prediction.astype(np.float32)
#             prediction = prediction.reshape(1,input_shape[1],66)
#             # current_prediction = model.predict(prediction)[0][0]
#             # print(prediction.shape)
#             model.set_tensor(input_details[0]['index'], prediction)
#             model.invoke()
#             output_data = model.get_tensor(output_details[0]['index'])
#             # predicted_class = np.argmax(output_data)


#             # print(output_data)
#             if output_data >=.89:
#                 exercise_ctr +=1
#                 temp_checking.append(one_exercise)

#                 # current_prediction = output_data
#                 # if prev_prediction + prediction_buffer_threshold >= current_prediction and prev_prediction - prediction_buffer_threshold <= current_prediction:
#                 #     None
#                 # else:
#                 #     exercise_ctr +=1

#                 # prev_prediction = current_prediction                
                

#             # interpreter.reset_all_variables()
        


#         # interpreter.set_tensor(input_details[0]['index'], [prediction])
#         # interpreter.invoke()
#         # output_data = interpreter.get_tensor(output_details[0]['index'])
#         # predicted_class = np.argmax(output_data)
#         # print("Predicted Class:", predicted_class)
#         # print("Predicted Class:", output_data)

        
#         # print(len(one_exercise))
#         # print(prediction.shape)
#         # 
#         # 
#         # if model.predict(prediction)[0][0]>=.8:
#         #     print('correct')
#     cv2.putText(frame, str(exercise_ctr), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
#     cv2.putText(frame, str(output_data), (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
#     mp_drawing.draw_landmarks(
#         frame,
#         results.pose_landmarks,
#         mp_pose.POSE_CONNECTIONS,
#         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
#         connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
#     )

#     cv2.imshow('MediaPipe Pose', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [49]:
# # add increase the max length by certain amount(for now 10)
# # soo it will always have more leeway when in the array
# def padding(pre_processed_input,optional_maxLength=0,increase_maxlen =.4): 
#     padded_sequences = []
#     if optional_maxLength != 0:
#         max_length = optional_maxLength
#     else:
#         max_length = max(len(sequence) for sequence in pre_processed_input)
#         max_length = max_length + (max_length *increase_maxlen )
    
#     for sequence in pre_processed_input:        
#         padding_length = max_length - len(sequence)
#         if padding_length >= 0:
#             padded_sequence = np.pad(sequence, ((0, padding_length), (0, 0)), mode='constant')
            
#         else:
#             padded_sequence = sequence[:max_length]
#         padded_sequences.append(padded_sequence)
#     padded_sequences = np.array(padded_sequences)

#     return padded_sequences


In [50]:
# do data augmentation that simulates the inferring process wherein it takes everyframes
# therfore we do not only append infront but also behind  with several number 
# we insert and simulate tha sequence in somewhere in between the max length not only at the beginning

In [51]:
# def streamlined_process(correct_execution,noise_data):
#     base_data = txt_pre_process(correct_execution,1,True,4)
#     base_data_noise = txt_pre_process(noise_data,0,True,4)

#     data = base_data[0]
#     data_noise = base_data_noise[0]

#     print('')
#     print('----------------------------correct data augmentation ----------------------------')
#     aug = common_length_sequence(data)
#     aug2 = apply_z_score(aug,1)
#     aug3 = paddingV2(aug2)
    
#     print('')
#     print('----------------------------data noise data augmentation ----------------------------')
#     aug_noise = paddingV2(data_noise,len(aug3[0]))
#     aug_noise = aug_noise[0:len(aug3)]


#     aug3_label = np.ones(len(aug3))
#     aug_noise_label = np.zeros(len(aug_noise))


#     rand_batches=concatenate_randomize_batches(aug3,aug3_label,aug_noise,aug_noise_label)
#     print('')
#     print('----------------------------final num set of sequences ----------------------------')
#     print('correct data ->',len(aug3))
#     print('wrong data ->',len(aug_noise))
#     print('combined and randomized data ->',rand_batches[0].shape)




#     X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.1, random_state=42)

#     # further splitting the dataset to have a set for simulation later on
#     X_train, predict_simulate_input, y_train, predict_simulate_label = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


#     model = Sequential()
#     model.add(LSTM(64, return_sequences=True, activation='relu', dropout=0.1, recurrent_dropout=0.1, input_shape=(len(aug3[0]), len(aug3[0][0]))))
#     model.add(LSTM(128, return_sequences=True, activation='relu')) 
#     model.add(LSTM(64, return_sequences=False, activation='relu')) 
#     model.add(Dense(64, activation='relu')) 
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy', 'binary_accuracy'])

#     model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))


#     return(aug3,aug_noise)



    



In [52]:
# streamlined_process('base(correct_execution).txt','base(wrong_execution).txt')

In [53]:
# print(len(aug3))

In [54]:
# for x in aug3[0]:
#     print(x[0])

In [55]:

# index = 5

# for x in range(len(testpop[index])):
#     print(testpop[index][x][0],'---',aug3[index][x][0])

In [56]:
# combined_inputs = np.concatenate((testpop,testpop2), axis = 0)

In [57]:
# testpop = np.array(testpop)
# testpop = testpop.reshape(-1,29,66)

In [58]:
# base_data = txt_pre_process('correct_new.txt',1,True,4)
# base_data_noise = txt_pre_process('wront_new.txt',0,True,4)

# temp_correct_start=[]
# temp_correct_wrong=[]

# data = base_data[0]
# data_noise = base_data_noise[0]

# print('')
# print('----------------------------correct data augmentation ----------------------------')
# aug = common_length_sequence(data)
# aug2 = apply_z_score(aug,1)
# aug3 = paddingV2(aug2)
# aug4 = populate_0_input(aug3,data_noise)
# aug4 = np.array(aug4)
# aug4 = aug4.reshape(-1,len(aug3[0]),len(aug3[0][0]))
# combined_inputs = np.concatenate((aug4,aug3), axis = 0)
# print('concat -> ', len(combined_inputs))


# print('')
# print('----------------------------data noise data augmentation ----------------------------')
# aug_noise_data1 = paddingV2(data_noise,len(aug3[0]))
# aug_noise_data2 = populate_0_input(aug_noise_data1,data_noise)
# aug_noise_data3 = aug_noise_data2[0:len(aug3)]
# aug_noise_data4 = np.array(aug_noise_data3)

# # for y in aug_noise_data4:
# #   for x in y:
# #     try:
# #         print('-->',len(x))
# #     except:
# #        print(x)
# #        break
# #     break

# # print(aug_noise_data4[0][4])




# print('ppppppp ->',len(aug_noise_data4))
# print('ppppppp ->',len(aug_noise_data4[0]))
# print('ppppppp ->',len(aug_noise_data4[0][0]))
# aug_noise_data5 = aug_noise_data4.reshape(-1,len(aug_noise_data4[0]),len(aug_noise_data4[0][0]))


# aug3_label = np.ones(len(aug3))
# aug_noise_label = np.zeros(len(aug_noise_data1))



# rand_batches=concatenate_randomize_batches(aug3,aug3_label,aug_noise_data5,aug_noise_label)

In [59]:
# len(testing_aug[0])

In [60]:
# model = tf.lite.Interpreter(model_path="D:\CLARK\Documents\RNN\RNN_proofOfConcept\converted_model_1069default9.tflite")
# base_data = txt_pre_process('base1234.txt',1,False,4)

# # 15


# model.allocate_tensors()
# input_details = model.get_input_details()
# output_details = model.get_output_details()
# input_shape = input_details[0]['shape']


# data = base_data[0]


# w_aug3=[]
# for x in data:
#     if (input_shape[1] - len(x))>0:
#         w_aug3.append(np.pad(x, ((0, input_shape[1] - len(x)), (0, 0)), mode='constant'))
#     else:
#         print('----')
#         continue






# ctr = 0
# for x in w_aug3:      
#     prediction = np.array(x)
#     prediction = prediction.astype(np.float32)
#     prediction = prediction.reshape(-1,input_shape[1],66)

#     model.set_tensor(input_details[0]['index'], prediction)
#     model.invoke()
#     output_data = model.get_tensor(output_details[0]['index'])
#     print(output_data[0][0])
#     if output_data[0][0] >= .99:
#         ctr = ctr + 1
#     else:
#         None
#         # print(output_data[0][0])
# print(ctr)





In [61]:
# w_base_data = txt_pre_process('wront_new.txt',1,True,4)
# w_base_data = w_base_data[0]
# w_aug = common_length_sequence(w_base_data)
# w_aug2 = apply_z_score(w_aug,1)
# w_aug3 = paddingV2(w_aug2)




In [62]:
# temp = []
# temp_compile = []
# len_seq = 29
# for x in range(len_seq):
#     temp.append(0)

# for x in range(len(all_array[0:120])):
#         temp.pop(0)
#         temp.append(all_array[x])

    
#     temp_compile.append(temp)
    

# print(len(temp_compile[0][0]))


In [63]:

# for u in range(20):
#     w_temp = []
#     for x in range(len(aug3[1])):  
#         w_temp.append(w_aug3[u][rand.randint(1,10)])



#     prediction = np.array(w_temp)
#     prediction = prediction.astype(np.float32)
#     prediction = prediction.reshape(1,input_shape[1],66)

#     model.set_tensor(input_details[0]['index'], prediction)
#     model.invoke()
#     output_data = model.get_tensor(output_details[0]['index'])
#     print(output_data)

In [64]:
# w_temp = []
# for x in range(len(aug3[1])):  
#     w_temp.append(w_aug3[0][rand.randint(1,10)])



# prediction = np.array(w_temp)
# prediction = prediction.astype(np.float32)
# prediction = prediction.reshape(1,input_shape[1],66)

# model.set_tensor(input_details[0]['index'], prediction)
# model.invoke()
# output_data = model.get_tensor(output_details[0]['index'])
# print(output_data)

In [65]:
# def populate_0_input(correct_data,noise_data):
#     index = 10
#     temp = []
#     temp_compilation = []
#     ctr = 0
#     rand_modifier =0
    
#     for set_sequence in correct_data:
#         rand_modifier = rand.randint(0,len(noise_data))

#         for x in range(len(aug3[1])):
#             ctr = ctr + 1 

#             if aug3[index][x][0] == 0:
#                 temp.append(noise_data[rand_modifier][rand.randint(1,len(noise_data[rand_modifier]))])
#             else:
#                 temp.append(set_sequence[x])

#             temp_compilation.append(temp)
#             temp =[]


#     # temp = np.array(temp)
#     # temp.shape

In [66]:



# prediction = np.array(aug3[x])
# prediction = prediction.astype(np.float32)
# prediction = prediction.reshape(-1,input_shape[1],66)

# model.set_tensor(input_details[0]['index'], prediction)
# model.invoke()
# output_data = model.get_tensor(output_details[0]['index'])
# if output_data[0][0] >= .99:
#     ctr = ctr + 1
# else:
#     None

In [67]:
# # this is inferencing with checkpoint

# import cv2
# import mediapipe as mp
# from IPython.display import clear_output
# temp_testing_int_1 = []
# temp_testing_int_2 = []

# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
# mp_drawing = mp.solutions.drawing_utils

# # cap = cv2.VideoCapture("vid4.mp4")
# cap = cv2.VideoCapture(0)


# # PARAMETERS:
# # ================================================================================================================================
# # timer to ready before posing/getting ready for the excercise
# set_timer_start = 50

# # ================================================================================================================================
# # range of how much change will it be considered a movement
# # lower -> small movement and jittery flaw of mediapipe will be detected
# # higher -> it wont detect immediately important frames
# # 0-1
# change_range = .03

# # ================================================================================================================================
# # a counter for the amount of time for the wait time before performing the exercise again
# between_exercise_ctr = 10



# isInitiated = False
# exercise_exectuted_ctr=0
# prev_x = 0
# prev_y = 0
# no_movement_ctr = 0
# movement_boolean = False
# timer = 0
# no_movement_threshold = 32
# for_prediction = []
# prev_array = []


# model = tf.lite.Interpreter(model_path="D:\CLARK\Documents\RNN\RNN_proofOfConcept\converted_model_whole_model4627(loss_0.029)(acc_1.0).tflite")



# model.allocate_tensors()
# input_details = model.get_input_details()
# output_details = model.get_output_details()
# input_shape = input_details[0]['shape']

# for x in range(33):
#     prev_array.append([0, 0])



# while True:
#     ret, frame = cap.read()
#     frame = cv2.flip(frame, 1)
#     image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = pose.process(image_rgb)

#     if not ret:
#         break

#     timer += 1 
#     landmark_all_present = False

#     if timer == set_timer_start:
#         file = open('base1234.txt', 'w')
#         if isInitiated==False:
#             file.write("START")
#             file.write('\n')
#         isInitiated = True

#     if results.pose_landmarks:
#         ctr_check=0

#         for idx, landmark in enumerate(results.pose_landmarks.landmark):
#             if idx == 32:
#                 landmark_all_present = "Landmarks -> ALL PRESENT"
#             else:
#                 landmark_all_present = "Landmarks -> SOME ARE MISSING"

#             x = landmark.x
#             y = landmark.y

# #checks if no movement is detected-----------------------------------------------------------------------------------------------------------------------------------------
#             if x >= (prev_array[idx][0] - change_range) and y <= (prev_array[idx][1] + change_range) and isInitiated:   
#                 # print(idx,")",x,"---",prev_array[idx][0],'---> NO MOVEMENT')
#                 ctr_check+=1          
                
#             else:
#                 # print(idx,")",x,"---",prev_array[idx][0],'--->YES MOVEMENT',)
#                 movement_boolean = True
#                 no_movement_ctr = 0

#             if ctr_check == no_movement_threshold:
#                 no_movement_ctr += 1 

#             if no_movement_ctr>=25:
#                 movement_boolean=False
#             prev_array[idx] = [x, y]
#         # print('counter->',no_movement_ctr)
#         # print(movement_boolean)

# #if movement is detected then it will record the coordinates into a text file-----------------------------------------------------------------------------------------------------
#         if movement_boolean:
#             text = "MOVEMENT"      
#             if timer >= set_timer_start and file is not None:
#                 for landmark in prev_array:
#                     # file.write("<")
#                     file.write("|")
#                     file.write(str(landmark[0]))
#                     file.write("|")
#                     file.write(str(landmark[1]))
#                     temp_testing_int_1.append(landmark[0])
#                     temp_testing_int_1.append(landmark[1])
#                     # file.write(">")
#                     # file.write("|")
#                 file.write('\n')
#                 temp_testing_int_2.append(temp_testing_int_1)
#                 temp_testing_int_1=[]

#         if no_movement_ctr >= 5:    
#             movement_boolean = False
#             text = "NO MOVEMENT!"

#         if no_movement_ctr == between_exercise_ctr:
#             print(len(temp_testing_int_2) ,'>',input_shape[1])
#             while len(temp_testing_int_2) > input_shape[1]:
#                 temp_testing_int_2.pop(0)
#                 # print('poping')

#             if len(temp_testing_int_2) <= input_shape[1]:
#                 # print('padding')
#                 temp_testing_int_2 = np.pad(temp_testing_int_2, ((0, input_shape[1] - len(temp_testing_int_2)), (0, 0)), mode='constant')

#             prediction = np.array(temp_testing_int_2)
#             print('===========',len(temp_testing_int_2))
#             print('===========',len(temp_testing_int_2[0]))
#             prediction = prediction.astype(np.float32)
#             prediction = prediction.reshape(1,input_shape[1],66)
#             # current_prediction = model.predict(prediction)[0][0]
#             # print(prediction.shape)
#             model.set_tensor(input_details[0]['index'], prediction)
#             model.invoke()
#             output_data = model.get_tensor(output_details[0]['index'])
#             print('--->',output_data)
#             clear_output(wait=True)
#             temp_testing_int_2=[]



            
            
#             exercise_exectuted_ctr+=1
#             file.write("END")  
#             file.write('\n')
#             file.write('\n')
#             file.write("START")
#             file.write('\n')



#         if timer >= set_timer_start and no_movement_ctr >=between_exercise_ctr:
#             cv2.putText(frame, "PERFORM THE EXERCISE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
# # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# #Outputing important informations-----------------------------------------------------------------------------------------------------------------------------------------------
#         cv2.putText(frame, str(movement_boolean), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, f"no_movement_ctr-> {no_movement_ctr}", (20, 37), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, str(landmark_all_present), (20, 54), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, f"timer-> {timer}", (20, 73), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
#         cv2.putText(frame, f"exercise exectuted-> {exercise_exectuted_ctr}", (20, 92), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
# # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#         if timer < set_timer_start:
#             cv2.putText(frame, "PREPARE YOUR STARTING POSE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
            

#         mp_drawing.draw_landmarks(
#             frame,
#             results.pose_landmarks,
#             mp_pose.POSE_CONNECTIONS,
#             landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
#             connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
#         )

#     cv2.imshow('MediaPipe Pose', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# if file is not None:
#     file.close()
# cap.release()
# cv2.destroyAllWindows()


In [68]:
# this is inferencing with checkpoint

import cv2
import mediapipe as mp
from IPython.display import clear_output
temp_testing_int_1 = []
temp_testing_int_2 = []


wrong_activator_line = []

for x in range(14):
    wrong_activator_line.append(0)

hand = []




# ---(for storing of sequences)---------------------------------------
left_upper_arm_temp = []
left_lower_arm_temp = []
left_hand_temp = []

right_upper_arm_temp = []
right_lower_arm_temp= []
right_hand_temp = []

left_upper_leg_temp = []
left_lower_leg_temp = []
left_feet_temp = []

right_upper_leg_temp = []
right_lower_leg_temp= []
right_feet_temp = []

body_temp = []
head_temp= []
# --------------------------------------------------------------------------------------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

temp_determiner = 0

# cap = cv2.VideoCapture("vid4.mp4")
cap = cv2.VideoCapture(0)


# PARAMETERS:
# ================================================================================================================================
# timer to ready before posing/getting ready for the excercise
set_timer_start = 50

# ================================================================================================================================
# range of how much change will it be considered a movement
# lower -> small movement and jittery flaw of mediapipe will be detected
# higher -> it wont detect immediately important frames
# 0-1
change_range = .03

# ================================================================================================================================
# a counter for the amount of time for the wait time before performing the exercise again
between_exercise_ctr = 15





isInitiated = False
exercise_exectuted_ctr=0
prev_x = 0
prev_y = 0
no_movement_ctr = 0
movement_boolean = False
timer = 0
no_movement_threshold = 32
for_prediction = []
prev_array = []




model = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\individual_models_1\otestingtesting(loss_0.063)(acc_0.982).tflite")


model_left_upper_arm = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_left_upper_arm6093(loss_0.049)(acc_0.984).tflite")
model_left_lower_arm = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_left_lower_arm6093(loss_0.137)(acc_0.963).tflite")
# model_left_hand = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\individual_models_1\\converted_model_5016left_hand.tflite")


model_right_upper_arm = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_right_upper_arm6093(loss_0.105)(acc_0.952).tflite")
model_right_lower_arm = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_right_lower_arm6093(loss_0.199)(acc_0.957).tflite")
# model_right_hand = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\individual_models_1\\converted_model_1334right_hand.tflite")


model_left_upper_leg = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_left_upper_leg6093(loss_0.083)(acc_0.973).tflite")
model_left_lower_leg = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_left_lower_leg6093(loss_0.046)(acc_0.989).tflite")
# model_left_feet = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\individual_models_1\\converted_model_5286left_feet.tflite")


model_right_upper_leg = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_right_upper_leg6093(loss_0.239)(acc_0.915).tflite")
model_right_lower_leg = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\updated_model\\converted_model_right_lower_leg6093(loss_0.57)(acc_0.84).tflite")
# model_right_feet = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\individual_models_1\\converted_model_6732right_feet.tflite")


# model_body = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\converted_model_9473left_upper_arm.tflite")
# model_head = tf.lite.Interpreter(model_path="D:\\CLARK\\Documents\\RNN\\RNN_proofOfConcept\\individual_models_1\\converted_model_7222head.tflite")

# base_data = txt_pre_process('base1234.txt',1,False,4)

# -----------model-------------------------
model.allocate_tensors()
input_details = model.get_input_details()
output_details = model.get_output_details()
input_shape = input_details[0]['shape']



# ===========================================================================================
# -----------left_upper_arm-------------------------
model_left_upper_arm.allocate_tensors()
input_details_left_upper_arm = model_left_upper_arm.get_input_details()
output_details_left_upper_arm = model_left_upper_arm.get_output_details()
input_shape_left_upper_arm = input_details_left_upper_arm[0]['shape']

# -----------left_lower_arm-------------------------
model_left_lower_arm.allocate_tensors()
input_details_left_lower_arm = model_left_lower_arm.get_input_details()
output_details_left_lower_arm = model_left_lower_arm.get_output_details()
input_shape_left_lower_arm = input_details_left_lower_arm[0]['shape']

# -----------left_hand-------------------------
# model_left_hand.allocate_tensors()
# input_details_left_hand = model_left_hand.get_input_details()
# output_details_left_hand = model_left_hand.get_output_details()
# input_shape_left_hand = input_details_left_hand[0]['shape']
# ===========================================================================================



# ===========================================================================================
# -----------right_upper_arm-------------------------
model_right_upper_arm.allocate_tensors()
input_details_right_upper_arm = model_right_upper_arm.get_input_details()
output_details_right_upper_arm = model_right_upper_arm.get_output_details()
input_shape_right_upper_arm = input_details_right_upper_arm[0]['shape']

# -----------right_lower_arm-------------------------
model_right_lower_arm.allocate_tensors()
input_details_right_lower_arm = model_right_lower_arm.get_input_details()
output_details_right_lower_arm = model_right_lower_arm.get_output_details()
input_shape_right_lower_arm = input_details_right_lower_arm[0]['shape']

# -----------right_hand-------------------------
# model_right_hand.allocate_tensors()
# input_details_right_hand = model_right_hand.get_input_details()
# output_details_right_hand = model_right_hand.get_output_details()
# input_shape_right_hand = input_details_right_hand[0]['shape']
# ===========================================================================================


# ===========================================================================================
# -----------left_upper_leg-------------------------
model_left_upper_leg.allocate_tensors()
input_details_left_upper_leg = model_left_upper_leg.get_input_details()
output_details_left_upper_leg = model_left_upper_leg.get_output_details()
input_shape_left_upper_leg = input_details_left_upper_leg[0]['shape']

# -----------left_lower_leg-------------------------
model_left_lower_leg.allocate_tensors()
input_details_left_lower_leg = model_left_lower_leg.get_input_details()
output_details_left_lower_leg = model_left_lower_leg.get_output_details()
input_shape_left_lower_leg = input_details_left_lower_leg[0]['shape']

# -----------left_feet-------------------------
# model_left_feet.allocate_tensors()
# input_details_left_feet = model_left_feet.get_input_details()
# output_details_left_feet = model_left_feet.get_output_details()
# input_shape_left_feet = input_details_left_feet[0]['shape']
# ===========================================================================================


# ===========================================================================================
# -----------right_upper_leg-------------------------
model_right_upper_leg.allocate_tensors()
input_details_right_upper_leg = model_right_upper_leg.get_input_details()
output_details_right_upper_leg = model_right_upper_leg.get_output_details()
input_shape_right_upper_leg = input_details_right_upper_leg[0]['shape']

# -----------right_lower_leg-------------------------
model_right_lower_leg.allocate_tensors()
input_details_right_lower_leg = model_right_lower_leg.get_input_details()
output_details_right_lower_leg = model_right_lower_leg.get_output_details()
input_shape_right_lower_leg = input_details_right_lower_leg[0]['shape']

# -----------right_feet-------------------------
# model_right_feet.allocate_tensors()
# input_details_right_feet = model_right_feet.get_input_details()
# output_details_right_feet = model_right_feet.get_output_details()
# input_shape_right_feet = input_details_right_feet[0]['shape']
# ===========================================================================================


# ===========================================================================================
# -----------body_model-------------------------
# model_body.allocate_tensors()
# input_details_body = model_body.get_input_details()
# output_details_body = model_body.get_output_details()
# input_shape_body = input_details_body[0]['shape']

# -----------head_model-------------------------
# model_head.allocate_tensors()
# input_details_head = model_head.get_input_details()
# output_details_head = model_head.get_output_details()
# input_shape_head = input_details_head[0]['shape']
# ===========================================================================================




left_upper_arm_compare = 0
left_lower_arm_compare = 0
left_hand_temp_compare = 0
right_upper_arm_compare = 0
right_lower_arm_compare = 0
right_hand_compare = 0
left_upper_leg_compare = 0
left_lower_leg_compare = 0
left_feet_compare = 0
right_upper_leg_compare = 0
right_lower_leg_compare = 0
right_feet_compare = 0
body_temp_compare = 0
head_temp_compare = 0

correct_threshold = 0

for x in range(33):
    prev_array.append([0, 0])



while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if not ret:
        break

    timer += 1 
    landmark_all_present = False

    if timer == set_timer_start:
        isInitiated = True


    if results.pose_landmarks:
        ctr_check=0

        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            if idx == 32:
                landmark_all_present = "Landmarks -> ALL PRESENT"
            else:
                landmark_all_present = "Landmarks -> SOME ARE MISSING"

            x = landmark.x
            y = landmark.y

#checks if no movement is detected-----------------------------------------------------------------------------------------------------------------------------------------
            if x >= (prev_array[idx][0] - change_range) and y <= (prev_array[idx][1] + change_range) and isInitiated:   
                ctr_check+=1          
                
            else:
                movement_boolean = True
                no_movement_ctr = 0

            if ctr_check == no_movement_threshold:
                no_movement_ctr += 1 

            if no_movement_ctr>=25:
                movement_boolean=False

            prev_array[idx] = [x, y]


#if movement is detected then it will record the coordinates into a text file-----------------------------------------------------------------------------------------------------
        if movement_boolean:    

            # temp_testing_int_2=[]
            # hand=[]

            # left_upper_arm_temp=[]
            # left_lower_arm_temp=[]
            # left_hand_temp=[]

            # right_upper_arm_temp=[]
            # right_lower_arm_temp=[]
            # right_hand_temp=[]

            # left_upper_leg_temp=[]
            # left_lower_leg_temp=[]
            # left_feet_temp=[]

            # right_upper_leg_temp=[]
            # right_lower_leg_temp=[]
            # right_feet_temp=[]

            # body_temp=[]
            # head_temp=[] 

            if timer >= set_timer_start :
                for landmark in prev_array:
                    temp_testing_int_1.append(landmark[0])
                    temp_testing_int_1.append(landmark[1])
                temp_testing_int_2.append(temp_testing_int_1)
                # hand.append([temp_testing_int_1[14],temp_testing_int_1[15],temp_testing_int_1[16],temp_testing_int_1[17],temp_testing_int_1[18],temp_testing_int_1[19],temp_testing_int_1[20],temp_testing_int_1[21]])
                # hand.append([temp_testing_int_1[22],temp_testing_int_1[23],temp_testing_int_1[26],temp_testing_int_1[27]])



# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                left_upper_arm_temp.append([temp_testing_int_1[22],temp_testing_int_1[23],temp_testing_int_1[26],temp_testing_int_1[27]])
                left_lower_arm_temp.append([temp_testing_int_1[26],temp_testing_int_1[27],temp_testing_int_1[30],temp_testing_int_1[31]])
                # left_hand_temp.append([temp_testing_int_1[30],temp_testing_int_1[31],temp_testing_int_1[34],temp_testing_int_1[35],temp_testing_int_1[38],temp_testing_int_1[39],temp_testing_int_1[42],temp_testing_int_1[43]])

                right_upper_arm_temp.append([temp_testing_int_1[24],temp_testing_int_1[25],temp_testing_int_1[28],temp_testing_int_1[29]])
                right_lower_arm_temp.append([temp_testing_int_1[28],temp_testing_int_1[29],temp_testing_int_1[32],temp_testing_int_1[33]])
                # right_hand_temp.append([temp_testing_int_1[32],temp_testing_int_1[33],temp_testing_int_1[36],temp_testing_int_1[37],temp_testing_int_1[40],temp_testing_int_1[41],temp_testing_int_1[44],temp_testing_int_1[45]])

                left_upper_leg_temp.append([temp_testing_int_1[46],temp_testing_int_1[47],temp_testing_int_1[50],temp_testing_int_1[51]])
                left_lower_leg_temp.append([temp_testing_int_1[50],temp_testing_int_1[51],temp_testing_int_1[54],temp_testing_int_1[55]])
                # left_feet_temp.append([temp_testing_int_1[54],temp_testing_int_1[55],temp_testing_int_1[58],temp_testing_int_1[59],temp_testing_int_1[62],temp_testing_int_1[63]])

                right_upper_leg_temp.append([temp_testing_int_1[48],temp_testing_int_1[49],temp_testing_int_1[52],temp_testing_int_1[53]])
                right_lower_leg_temp.append([temp_testing_int_1[52],temp_testing_int_1[53],temp_testing_int_1[56],temp_testing_int_1[57]])
                # right_feet_temp.append([temp_testing_int_1[56],temp_testing_int_1[57],temp_testing_int_1[60],temp_testing_int_1[61],temp_testing_int_1[64],temp_testing_int_1[65]])

                # body_temp.append([temp_testing_int_1[22],temp_testing_int_1[23],temp_testing_int_1[24],temp_testing_int_1[25],temp_testing_int_1[46],temp_testing_int_1[47],temp_testing_int_1[48],temp_testing_int_1[49]])
                # head_temp.append([temp_testing_int_1[14],temp_testing_int_1[15],temp_testing_int_1[16],temp_testing_int_1[17],temp_testing_int_1[18],temp_testing_int_1[19],temp_testing_int_1[20],temp_testing_int_1[21]])
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


                temp_testing_int_1=[]

        if no_movement_ctr >= 5:    
            movement_boolean = False
            text = "NO MOVEMENT!"

        if no_movement_ctr == between_exercise_ctr:
            while len(temp_testing_int_2) > input_shape[1]:
                temp_testing_int_2.pop(0)
                # hand.pop(0)

                left_upper_arm_temp.pop(0)
                left_lower_arm_temp.pop(0)
                # left_hand_temp.pop(0)

                right_upper_arm_temp.pop(0)
                right_lower_arm_temp.pop(0)
                # right_hand_temp.pop(0)

                left_upper_leg_temp.pop(0)
                left_lower_leg_temp.pop(0)
                # left_feet_temp.pop(0)

                right_upper_leg_temp.pop(0)
                right_lower_leg_temp.pop(0)
                # right_feet_temp.pop(0)

                # body_temp.pop(0)
                # head_temp.pop(0)

                
            temp_determiner = len(left_upper_arm_temp)-1

            if len(temp_testing_int_2) <= input_shape[1]:
                print('input_shape->',input_shape[1])
                print('input_shape->',temp_testing_int_2)
                temp_testing_int_2 = np.pad(temp_testing_int_2, ((0, input_shape[1] - len(temp_testing_int_2)), (0, 0)), mode='constant')

# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                temp_determiner = len(left_upper_arm_temp)-1

                print('test1->',input_shape_left_upper_arm[1])
                print('test1->',len(left_upper_arm_temp))

            if len(left_upper_arm_temp) <= input_shape_left_upper_arm[1]:
                left_upper_arm_temp = np.pad(left_upper_arm_temp, ((0, input_shape_left_upper_arm[1] - len(left_upper_arm_temp)), (0, 0)), mode='constant')
                left_lower_arm_temp = np.pad(left_lower_arm_temp, ((0, input_shape_left_lower_arm[1] - len(left_lower_arm_temp)), (0, 0)), mode='constant')
                # left_hand_temp = np.pad(left_hand_temp, ((0, input_shape_left_hand[1] - len(left_hand_temp)), (0, 0)), mode='constant')

                right_upper_arm_temp = np.pad(right_upper_arm_temp, ((0, input_shape_right_upper_arm[1] - len(right_upper_arm_temp)), (0, 0)), mode='constant')
                right_lower_arm_temp = np.pad(right_lower_arm_temp, ((0, input_shape_right_lower_arm[1] - len(right_lower_arm_temp)), (0, 0)), mode='constant')
                # right_hand_temp = np.pad(right_hand_temp, ((0, input_shape_right_hand[1] - len(right_hand_temp)), (0, 0)), mode='constant')

                left_upper_leg_temp = np.pad(left_upper_leg_temp, ((0, input_shape_left_upper_leg[1] - len(left_upper_leg_temp)), (0, 0)), mode='constant')
                left_lower_leg_temp = np.pad(left_lower_leg_temp, ((0, input_shape_left_lower_leg[1] - len(left_lower_leg_temp)), (0, 0)), mode='constant')
                # left_feet_temp = np.pad(left_feet_temp, ((0, input_shape_left_feet[1] - len(left_feet_temp)), (0, 0)), mode='constant')

                right_upper_leg_temp = np.pad(right_upper_leg_temp, ((0, input_shape_right_upper_leg[1] - len(right_upper_leg_temp)), (0, 0)), mode='constant')
                right_lower_leg_temp = np.pad(right_lower_leg_temp, ((0, input_shape_right_lower_leg[1] - len(right_lower_leg_temp)), (0, 0)), mode='constant')
                # right_feet_temp = np.pad(right_feet_temp, ((0, input_shape_right_feet[1] - len(right_feet_temp)), (0, 0)), mode='constant')

                # body_temp = np.pad(body_temp, ((0, input_shape_body[1] - len(body_temp)), (0, 0)), mode='constant')
                # head_temp = np.pad(head_temp, ((0, input_shape_head[1] - len(head_temp)), (0, 0)), mode='constant')
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

            prediction = np.array(temp_testing_int_2)
            prediction = prediction.astype(np.float32)
            prediction = prediction.reshape(1,input_shape[1],input_shape[2])
            model.set_tensor(input_details[0]['index'], prediction)
            model.invoke()            
            output_data = model.get_tensor(output_details[0]['index'])
            print('whole --->',output_data)
          

            # model_inference(hand,model_hand,input_shape_hand,input_details_hand,output_details_hand,'left_upper_arm')
# ---------------------------------------------------------------------------------------------------------------------------------------------------------
            correct_threshold = .85           
            left_upper_arm_compare = model_inference(left_upper_arm_temp,model_left_upper_arm,input_shape_left_upper_arm,input_details_left_upper_arm,output_details_left_upper_arm,'left_upper_arm') 
            left_lower_arm_compare =  model_inference(left_lower_arm_temp,model_left_lower_arm,input_shape_left_lower_arm,input_details_left_lower_arm,output_details_left_lower_arm,'left_lower_arm')
            # left_hand_temp_compare = model_inference(left_hand_temp,model_left_hand,input_shape_left_hand,input_details_left_hand,output_details_left_hand,'left_hand')


            right_upper_arm_compare = model_inference(right_upper_arm_temp,model_right_upper_arm,input_shape_right_upper_arm,input_details_right_upper_arm,output_details_right_upper_arm,'right_upper_arm')                     
            right_lower_arm_compare = model_inference(right_lower_arm_temp,model_right_lower_arm,input_shape_right_lower_arm,input_details_right_lower_arm,output_details_right_lower_arm,'right_lower_arm')
            # right_hand_compare =  model_inference(right_hand_temp,model_right_hand,input_shape_right_hand,input_details_right_hand,output_details_right_hand,'right_hand')>=correct_threshold:

            

            left_upper_leg_compare =  model_inference(left_upper_leg_temp,model_left_upper_leg,input_shape_left_upper_leg,input_details_left_upper_leg,output_details_left_upper_leg,'left_upper_leg')
            left_lower_leg_compare =  model_inference(left_lower_leg_temp,model_left_lower_leg,input_shape_left_lower_leg,input_details_left_lower_leg,output_details_left_lower_leg,'left_lower_leg')
            # left_feet_compare = model_inference(left_feet_temp,model_left_feet,input_shape_left_feet,input_details_left_feet,output_details_left_feet,'left_feet')>=correct_threshold:


            right_upper_leg_compare =  model_inference(right_upper_leg_temp,model_right_upper_leg,input_shape_right_upper_leg,input_details_right_upper_leg,output_details_right_upper_leg,'right_upper_leg')
            right_lower_leg_compare =  model_inference(right_lower_leg_temp,model_right_lower_leg,input_shape_right_lower_leg,input_details_right_lower_leg,output_details_right_lower_leg,'right_lower_leg')
            # right_feet_compare =  model_inference(right_feet_temp,model_right_feet,input_shape_right_feet,input_details_right_feet,output_details_right_feet,'right_feet')>=correct_threshold:



            # body_temp_compare =  model_inference(body_temp,model_body,input_shape_body,input_details_body,output_details_body,'body_model'):           
            # head_temp_compare =  model_inference(head_temp,model_head,input_shape_head,input_details_head,output_details_head,'head_model'):


            clear_output(wait=True)
            temp_testing_int_2=[]
            hand=[]

            left_upper_arm_temp=[]
            left_lower_arm_temp=[]
            left_hand_temp=[]

            right_upper_arm_temp=[]
            right_lower_arm_temp=[]
            right_hand_temp=[]

            left_upper_leg_temp=[]
            left_lower_leg_temp=[]
            left_feet_temp=[]

            right_upper_leg_temp=[]
            right_lower_leg_temp=[]
            right_feet_temp=[]

            body_temp=[]
            head_temp=[]             
            exercise_exectuted_ctr+=1


        if timer >= set_timer_start and no_movement_ctr >=between_exercise_ctr:
            cv2.putText(frame, "PERFORM THE EXERCISE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 150, 255), 2)
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    

# ====================================================(left_arm)=========================================================================
# --------------------------------------------------(left_upper_arm)-----------------------------------------------------------------------
    correct_threshold_color_modifier = .25
    if left_upper_arm_compare >= correct_threshold:
        color_percentage_modifier = (left_upper_arm_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100          
        draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(int(255*color_percentage_modifier), 0,0 ))        
    elif left_upper_arm_compare <= correct_threshold: 
        draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(0, 255, 0))
# ---------------------------------------------------(left_lower_arm)----------------------------------------------------------------------
    if left_lower_arm_compare >= correct_threshold: 
        color_percentage_modifier = (left_lower_arm_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100    
        draw_pose_lines(frame,[prev_array[13][0],prev_array[13][1],prev_array[15][0],prev_array[15][1]],(int(255*color_percentage_modifier), 0,0 ))
    elif left_lower_arm_compare <= correct_threshold: 
        draw_pose_lines(frame,[prev_array[13][0],prev_array[13][1],prev_array[15][0],prev_array[15][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[13][0],prev_array[13][1],prev_array[15][0],prev_array[15][1]],(0, 255, 0))
# ----------------------------------------------------(left_hand)---------------------------------------------------------------------
    # if left_hand_temp_compare >= correct_threshold:  
        # draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]])
        # wrong_activator_line[2]=0
    # elif left_hand_temp_compare <= correct_threshold: 
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(0, 0, 255))
    # wrong_activator_line[2]=0
    # else: 
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(255, 0, 0))
# =============================================================================================================================



# ===============================================(right_arm)==============================================================================
# --------------------------------------------------(right_upper_arm)-----------------------------------------------------------------------
    if right_upper_arm_compare >= correct_threshold: 
        color_percentage_modifier = (right_upper_arm_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100  
        draw_pose_lines(frame,[prev_array[12][0],prev_array[12][1],prev_array[14][0],prev_array[14][1]],(255,int(255*color_percentage_modifier),0 ))
    elif right_upper_arm_compare <= correct_threshold: 
        draw_pose_lines(frame,[prev_array[12][0],prev_array[12][1],prev_array[14][0],prev_array[14][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[12][0],prev_array[12][1],prev_array[14][0],prev_array[14][1]],(0, 255, 0))
# ----------------------------------------------------(right_lower_arm---------------------------------------------------------------------
    if right_lower_arm_compare >= correct_threshold:
        color_percentage_modifier = (right_lower_arm_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100  
        draw_pose_lines(frame,[prev_array[14][0],prev_array[14][1],prev_array[16][0],prev_array[16][1]],(255, int(255*color_percentage_modifier),0 ))
    elif right_lower_arm_compare <= correct_threshold:
        draw_pose_lines(frame,[prev_array[14][0],prev_array[14][1],prev_array[16][0],prev_array[16][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[14][0],prev_array[14][1],prev_array[16][0],prev_array[16][1]],(0, 255, 0))
# -----------------------------------------------------(right_hand)--------------------------------------------------------------------
    # if right_hand_compare >= correct_threshold:
        # draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]])
        # wrong_activator_line[5]=0
    # elif right_hand_compare <= correct_threshold:
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(0, 0, 255))
        # wrong_activator_line[5]=0
    # else: 
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(255, 0, 0))
# =============================================================================================================================




# =================================================(left_leg)============================================================================
# ----------------------------------------------------(left_upper_leg)---------------------------------------------------------------------
    if left_upper_leg_compare >= correct_threshold:
        color_percentage_modifier = (left_upper_leg_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100  
        draw_pose_lines(frame,[prev_array[23][0],prev_array[23][1],prev_array[25][0],prev_array[25][1]],(255,int(255*color_percentage_modifier),0 ))
    elif left_upper_leg_compare <= correct_threshold:
        draw_pose_lines(frame,[prev_array[23][0],prev_array[23][1],prev_array[25][0],prev_array[25][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[23][0],prev_array[23][1],prev_array[25][0],prev_array[25][1]],(0, 255, 0))
# -----------------------------------------------------(left_lower_leg)--------------------------------------------------------------------
    if left_lower_leg_compare >= correct_threshold:
        color_percentage_modifier = (left_lower_leg_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100  
        draw_pose_lines(frame,[prev_array[25][0],prev_array[25][1],prev_array[27][0],prev_array[27][1]],(255,int(255*color_percentage_modifier),0 ))
    elif left_lower_leg_compare <= correct_threshold:
        draw_pose_lines(frame,[prev_array[25][0],prev_array[25][1],prev_array[27][0],prev_array[27][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[25][0],prev_array[25][1],prev_array[27][0],prev_array[27][1]],(0, 255, 0))
# -------------------------------------------------------(left_feet)------------------------------------------------------------------
    # if left_feet_compare >= correct_threshold:
        # draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]])
    # elif left_feet_compare <= correct_threshold:
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(0, 0, 255))
    # else: 
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(255, 0, 0))
# =============================================================================================================================


# =======================================================(right_leg)======================================================================
# ----------------------------------------------------(left_upper_leg)---------------------------------------------------------------------
    if right_upper_leg_compare >= correct_threshold:
        color_percentage_modifier = (right_upper_leg_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100  
        draw_pose_lines(frame,[prev_array[24][0],prev_array[24][1],prev_array[26][0],prev_array[26][1]],(255,int(255*color_percentage_modifier),0 ))
    elif right_upper_leg_compare <= correct_threshold:
        draw_pose_lines(frame,[prev_array[24][0],prev_array[24][1],prev_array[26][0],prev_array[26][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[24][0],prev_array[24][1],prev_array[26][0],prev_array[26][1]],(0, 255, 0))
# ----------------------------------------------------(left_upper_leg)---------------------------------------------------------------------
    if right_lower_leg_compare >= correct_threshold:
        color_percentage_modifier = (right_lower_leg_compare - correct_threshold-correct_threshold_color_modifier) / (1 - correct_threshold-correct_threshold_color_modifier ) * 100  
        draw_pose_lines(frame,[prev_array[26][0],prev_array[26][1],prev_array[28][0],prev_array[28][1]],(255,int(255*color_percentage_modifier),0 ))
    elif right_lower_leg_compare <= correct_threshold:
        draw_pose_lines(frame,[prev_array[26][0],prev_array[26][1],prev_array[28][0],prev_array[28][1]],(0, 0, 255))
    else: 
        draw_pose_lines(frame,[prev_array[26][0],prev_array[26][1],prev_array[28][0],prev_array[28][1]],(0, 255, 0))
# ----------------------------------------------------(left_upper_leg)---------------------------------------------------------------------
    # if right_feet_compare >= correct_threshold:
    #     # draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]])
    #   wrong_activator_line[3]=0
    # elif right_feet_compare <= correct_threshold:
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(0, 0, 255))
    #       wrong_activator_line[3]=0
    # else: 
    #     draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]],(255, 0, 0))
# =============================================================================================================================



# =============================================================================================================================   
    # if body_temp_compare >= correct_threshold:
        # draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]])

    # if head_temp_compare >= correct_threshold:
        # draw_pose_lines(frame,[prev_array[11][0],prev_array[11][1],prev_array[13][0],prev_array[13][1]])
# =============================================================================================================================



#Outputing important informations-----------------------------------------------------------------------------------------------------------------------------------------------
        cv2.putText(frame, str(movement_boolean), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"no_movement_ctr-> {no_movement_ctr}", (20, 37), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, str(landmark_all_present), (20, 54), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"timer-> {timer}", (20, 73), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
        cv2.putText(frame, f"exercise exectuted-> {exercise_exectuted_ctr}", (20, 92), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 1)
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        if timer < set_timer_start:
            cv2.putText(frame, "PREPARE YOUR STARTING POSE", (180, 460), cv2.FONT_HERSHEY_SIMPLEX, .5, (0, 0, 255), 2)
            


        # mp_drawing.draw_landmarks(
        #     frame,
        #     results.pose_landmarks,
        #     mp_pose.POSE_CONNECTIONS,
        #     landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
        #     connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
        # )

    cv2.imshow('MediaPipe Pose', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [69]:

# if temp = 0 >=90:
#     print('yesysey')

In [70]:
# temp=[2,3,4,5,6,7]
# temp = np.array(temp)
# print(temp[0])

In [71]:
# # inferencing



# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
# mp_drawing = mp.solutions.drawing_utils

# # cap = cv2.VideoCapture("vid4.mp4")
# cap = cv2.VideoCapture(0)
# frame_ctr = 0
# frame_ctr_threshold = 0
# exercise_ctr = 0
# prev_prediction = 0
# output_data = 0
# temp_checking = []
# all_array = []
# # it means it is waiting to reset
# prediction_buffer_threshold = .05
# model = tf.lite.Interpreter(model_path="D:\CLARK\Documents\RNN\RNN_proofOfConcept\converted_model_9683default3.tflite")
# model.allocate_tensors()
# input_details = model.get_input_details()
# output_details = model.get_output_details()
# input_shape = input_details[0]['shape']
# # print('input ->',)

# prediction =[]
# one_exercise = []
# # max length of sequence
# for y in range(input_shape[1]):
#     sequence_input=[]
#     # max num of landmark x and y flattened
#     for x in range(66):
#         sequence_input.append(0)
#     one_exercise.append(sequence_input)
    



# while True:
#     frame_ctr += 1
#     # print(frame_ctr)
#     ret, frame = cap.read()
#     frame = cv2.flip(frame, 1)
#     image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = pose.process(image_rgb)

#     if not ret:
#         break




#     if results.pose_landmarks:
#         temp_sequence=[]
#         for idx, landmark in enumerate(results.pose_landmarks.landmark):
#             x = landmark.x
#             y = landmark.y
#             # print(idx,")",x,"---",prev_array[idx][0],)
#             # print(idx)
#             # x = round(float(x),4)
#             # y = round(float(y),4)
#             temp_sequence.append(x)
#             temp_sequence.append(y)
#         one_exercise.append(temp_sequence)
#         print(one_exercise[0][0],'------',temp_sequence[0])
        

#     # if max length of the exercise sequence was reached then pop it pop off
#         if len(one_exercise) >= input_shape[1]:
#             one_exercise.pop(0)

#         all_array.append(one_exercise[:])
        
#         if frame_ctr >= frame_ctr_threshold:            
#             frame_ctr = 0
            
#             prediction = np.array(one_exercise)
#             prediction = prediction.astype(np.float32)
#             prediction = prediction.reshape(1,input_shape[1],66)
#             # current_prediction = model.predict(prediction)[0][0]
#             # print(prediction)
#             model.set_tensor(input_details[0]['index'], prediction)
#             model.invoke()
#             output_data = model.get_tensor(output_details[0]['index'])
#             # predicted_class = np.argmax(output_data)


#             # print(output_data)
#             if output_data >=.89:
#                 exercise_ctr +=1
#                 temp_checking.append(one_exercise)

#                 # current_prediction = output_data
#                 # if prev_prediction + prediction_buffer_threshold >= current_prediction and prev_prediction - prediction_buffer_threshold <= current_prediction:
#                 #     None
#                 # else:
#                 #     exercise_ctr +=1

#                 # prev_prediction = current_prediction                
                

#             # interpreter.reset_all_variables()
        


#         # interpreter.set_tensor(input_details[0]['index'], [prediction])
#         # interpreter.invoke()
#         # output_data = interpreter.get_tensor(output_details[0]['index'])
#         # predicted_class = np.argmax(output_data)
#         # print("Predicted Class:", predicted_class)
#         # print("Predicted Class:", output_data)

        
#         # print(len(one_exercise))
#         # print(prediction.shape)
#         # 
#         # 
#         # if model.predict(prediction)[0][0]>=.8:
#         #     print('correct')
#     cv2.putText(frame, str(exercise_ctr), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
#     cv2.putText(frame, str(output_data), (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
#     mp_drawing.draw_landmarks(
#         frame,
#         results.pose_landmarks,
#         mp_pose.POSE_CONNECTIONS,
#         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
#         connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
#     )

#     cv2.imshow('MediaPipe Pose', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [72]:
# import cv2
# import mediapipe as mp

# # Initialize MediaPipe Pose model
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# # Start video capture
# cap = cv2.VideoCapture(0)

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Process the frame with MediaPipe
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = pose.process(frame_rgb)

#     # Draw only specific connections and landmarks
#     if results.pose_landmarks:
#         # Draw specific connections (e.g., from left shoulder to left wrist)
#         specific_connections = [(mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_WRIST.value)]
#         for connection in specific_connections:
#             idx_start, idx_end = connection
#             start_point = (int(results.pose_landmarks.landmark[idx_start].x * frame.shape[1]),
#                            int(results.pose_landmarks.landmark[idx_start].y * frame.shape[0]))
#             end_point = (int(results.pose_landmarks.landmark[idx_end].x * frame.shape[1]),
#                          int(results.pose_landmarks.landmark[idx_end].y * frame.shape[0]))
#             cv2.line(frame, start_point, end_point, (0, 0, 255), 2)

#         # Draw specific landmarks (e.g., left wrist)
#         specific_landmarks = [mp_pose.PoseLandmark.LEFT_WRIST]
#         for landmark in specific_landmarks:
#             idx = landmark.value
#             center_point = (int(results.pose_landmarks.landmark[idx].x * frame.shape[1]),
#                             int(results.pose_landmarks.landmark[idx].y * frame.shape[0]))
#             cv2.circle(frame, center_point, radius=2, color=(0, 255, 0), thickness=2)

#     # Display the frame with specific connections and landmarks
#     cv2.imshow('Customized Frame', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()
